### Data Ingestion


In [ ]:
### Document Data Structure

from langchain_core.documents import Document

Document(page_content="ok", metadata={})

In [ ]:
doc=Document(
    page_content="this is the main content I am using to create a RAG",
    metadata={
        "source": "example.txt",
        "pages": 1,
        "author": "Anshuman",
        "date_created": "2026-01-27"
    }
)
doc

In [ ]:
### create a simple txt File
import os
os.makedirs('../data/text_files', exist_ok=True)

In [ ]:
sample_text = {
    '../data/text_files/python_intro.txt':"""python programming introduction"""
}
for filepath, content in sample_text.items():
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)

print("Sample txt file created!!!")

In [ ]:
### TextLoader
from langchain_community.document_loaders import TextLoader

loader = TextLoader('../data/text_files/python_intro.txt', encoding='utf-8')
document = loader.load()
print(document)

In [ ]:
### Directory Loader
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader = DirectoryLoader(
    '../data/text_files',
    glob='**/*.txt', ## pattern to match the files
    loader_cls=TextLoader, ##loader class to use
    loader_kwargs={'encoding': 'utf-8'},
    show_progress=False
)

documents=dir_loader.load()
documents

In [ ]:
### PDF files
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

### Directory Loader
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader = DirectoryLoader(
    '../data/pdf_files',
    glob='**/*.pdf', ## pattern to match the files
    loader_cls=PyMuPDFLoader, ##loader class to use
    show_progress=False
)

pdf_documents=dir_loader.load()
pdf_documents

In [ ]:
type(pdf_documents[0])

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [ ]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

In [ ]:
all_pdf_documents

In [ ]:
### Text splitting get into chunks

from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents,chunk_size=800,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [ ]:
chunks=split_documents(all_pdf_documents)
chunks

### Embedding and VectorStoreDB


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    def __init__(self, model_name: str = "all-miniLM-L6-v2"):
        """
        Initialise the embedding manager

        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    
    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model Loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts

        Args: 
            texts: List of text strings to embed

            Returns: numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded!!!")
        
        print(f"Generating embeddings for {len(texts)} text...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    
    ## initialise the embedding manager
embedding_manager=EmbeddingManager()
embedding_manager

### Vector Store


In [ ]:
class VectorStore:
    """Manage document embeddings in a chromaDB vector store """

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = '../data/vector_store'):
        """
        Initialise the vector store 

        Args:
            collection_name: Name of the chromaDB collection 
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Intialise chromaDB client and collection"""   
        try: 
            # create persistent chromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector Store initialised. Collection: {self.collection_name}")
            print(f"Existing documents in Collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initialsing vector store {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args: 
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match the number of embeddings!!!")
        
        print(f"Adding {len(documents)} documents to vector store...")

        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list =[]

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID 
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            # Document content
            documents_text.append(doc.page_content)

            # Embedding
            embeddings_list.append(embedding.tolist())

        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

In [ ]:
### Conver the text to embeddings
texts=[doc.page_content for doc in chunks]
# texts ---- commented out

## Generate the embeddings
embeddings=embedding_manager.generate_embeddings(texts)

## store in the vectorDB
vectorstore.add_documents(chunks, embeddings)

### Retriever Pipeline from VectorStore


In [ ]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""

    def __init__(self, vectorstore: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialise the retriever

        Args: 
            vector_store: vectorstore containing document embeddings
            embedding_manager: Manager for generating query embeeddings
        """

        self.vectorstore = vectorstore
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 10, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: the search query 
            top_k: number of top results to return 
            score_threshold: minimum simialarity score threshold 

        Returns:
            list of dictionaries containing retreieved documents and metadata 
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threhold: {score_threshold}")

        # generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vectorstore.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            # Process results

            retrieved_documents = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)

                    similarity_score = 1 - distance

                    if similarity_score >= score_threshold:
                        retrieved_documents.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })

                print(f"Retrieved {len(retrieved_documents)} documents (after filtering)")
            else:
                print(f"No documents found")

            return retrieved_documents
        
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return[]

rag_retriever=RAGRetriever(vectorstore, embedding_manager)


In [ ]:
rag_retriever

In [ ]:
rag_retriever.retrieve("Technical skills languages")

### Integration Of VectorDB Context Pipeline with LLM Output


In [ ]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialise Groq LLM
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.1-8b-instant", temperature=0.1, max_tokens=1024)
# llm=ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.3-70b-versatile", temperature=0.1, max_tokens=1024)

### Simple RAG Function: retrieve context + generate response
def rag_simple(query, retriever, llm, top_k=3):
    ## retrieve the context
    results=retriever.retrieve(query, top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "NO Relevant context found to answer the question!!!"

    ## Generate the answer using the LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context, query=query)])
    return response.content

In [ ]:
answer=rag_simple("what are anshuman's technical skills?", rag_retriever, llm)

In [ ]:
print(answer)

In [ ]:
new_answer=rag_simple("what is anshuman's cgpa or gpa?", rag_retriever, llm)
print(new_answer)

#### Enhanced RAG Pipeline Features ----


In [ ]:
# --- Enhanced RAG pipeline features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
        - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)

    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])

    # Generate answer
    prompt = f"""Use the following context to answer the question concisely. \nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response=llm.invoke([prompt.format(context=context, query=query)])

    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

In [ ]:
# Example Usage
result = rag_advanced("Summarise the whole resume of anshuman with his skills", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer: ", result['answer'])
print("Sources: ", result['sources'])
print("Confidence: ", result['confidence'])
print("Context Preview: ", result['context'][:300])